In [1]:
import Pkg
Pkg.activate(".")

  Activating project at `~/Developer/DistributedStreams`


In [2]:
using Distributed, DistributedStreams

In [3]:
ser_buf = DistributedStreams.Serialize.serialize((a="this is fun!", b=true, c=1, d=1.2, e=1//2))

104-element Vector{UInt8}:
 0x78
 0x9c
 0x33
 0xf7
 0xf2
 0x91
 0x60
 0x61
 0x60
 0x60
 0x30
 0x11
 0x60
    ⋮
 0x37
 0xd1
 0x61
 0x66
 0x78
 0xf0
 0x10
 0x00
 0xea
 0xd6
 0x19
 0xd9

In [4]:
bit_buf = DistributedStreams.Bits.serialize((a="this is fun!", b=true, c=1, d=1.2, e=1//2))

99-element Vector{UInt8}:
 0x78
 0x9c
 0x63
 0x60
 0x80
 0x00
 0xb5
 0x2a
 0x08
 0xcd
 0x0a
 0xe5
 0x33
    ⋮
 0x01
 0x15
 0x02
 0x2e
 0x61
 0x82
 0xd2
 0x00
 0xe0
 0xa9
 0x1d
 0xd5

In [5]:
DistributedStreams.Serialize.deserialize(ser_buf)

(a = "this is fun!", b = true, c = 1, d = 1.2, e = 1//2)

In [6]:
DistributedStreams.Bits.deserialize(bit_buf)

(a = "this is fun!", b = true, d = 1.2, e = 1//2, c = 1)

In [7]:
String(ser_buf)

"x\x9c3\xf7\xf2\x91`a``0\x11`\xe4\xf2K\xccMM\t)-\xc8I\x95\xf7\x9b\xed\xc2\x04\x14\x15aMK\xcf\xc8\xcc\x12\b\x06\t\xb0\x02\x05\x18\x14]\x198\x18\xf8\x04\x189\x82\x12K2\xf3\xf3\x12s\xe4\xfd\xe6\xb90\x82d8,\x15yJ22\x8b\x15\x80(\xad4O\xd1\xf7\x01\x9f1\x18|\xb67\xd1afx\xf0\x10\0\xea\xd6\x19\xd9"

In [8]:
String(bit_buf)

"x\x9cc`\x80\0\xb5*\b\xcd\n\xe53B\xf9\x8cP\xbe:\x03*\x9f\xf1\xfa~\xc1o\xf5\f\f\x89<h\n`|ƫ\x10\x05%\x19\x99\xc5\n@\x94V\x9a\xa7HȰ\$Ft\tt\x9a\x90\t\xc9\x1cP\x01.B&\x110(\x05f\x10\x1f\x0e\x8d\xc6`\xf0ٞ\x90A\xa9\x02P\x01\x15\x02.a\x82\xd2\0\xe0\xa9\x1d\xd5"

In [9]:
addprocs(1)

1-element Vector{Int64}:
 2

In [10]:
@everywhere using DistributedStreams

In [11]:
fn(x) = x+1

fn (generic function with 1 method)

In [12]:
workers()

1-element Vector{Int64}:
 2

In [13]:
x = remotecall(fn, 2, 1)

Future(2, 1, 5, ReentrantLock(nothing, 0x00000000, 0x00, Base.GenericCondition{Base.Threads.SpinLock}(Base.IntrusiveLinkedList{Task}(nothing, nothing), Base.Threads.SpinLock(0)), (0, 0, 0)), nothing)

In [14]:
q = try
    fetch(x)
catch e
    e
end

RemoteException(2, CapturedException(UndefVarError(Symbol("#fn")), Any[(deserialize_datatype at Serialization.jl:1399, 1), (handle_deserialize at Serialization.jl:867, 1), (deserialize at Serialization.jl:814, 1), (handle_deserialize at Serialization.jl:874, 1), (deserialize at Serialization.jl:814 [inlined], 1), (deserialize_msg at messages.jl:87, 1), (#invokelatest#2 at essentials.jl:887 [inlined], 1), (invokelatest at essentials.jl:884 [inlined], 1), (message_handler_loop at process_messages.jl:176, 1), (process_tcp_streams at process_messages.jl:133, 1), (#103 at process_messages.jl:121, 1)]))

In [15]:
q

RemoteException(2, CapturedException(UndefVarError(Symbol("#fn")), Any[(deserialize_datatype at Serialization.jl:1399, 1), (handle_deserialize at Serialization.jl:867, 1), (deserialize at Serialization.jl:814, 1), (handle_deserialize at Serialization.jl:874, 1), (deserialize at Serialization.jl:814 [inlined], 1), (deserialize_msg at messages.jl:87, 1), (#invokelatest#2 at essentials.jl:887 [inlined], 1), (invokelatest at essentials.jl:884 [inlined], 1), (message_handler_loop at process_messages.jl:176, 1), (process_tcp_streams at process_messages.jl:133, 1), (#103 at process_messages.jl:121, 1)]))

In [16]:
typeof(q)

RemoteException

In [17]:
@everywhere fn(x) = x+1

In [18]:
x = remotecall(fn, 2, 1)

Future(2, 1, 9, ReentrantLock(nothing, 0x00000000, 0x00, Base.GenericCondition{Base.Threads.SpinLock}(Base.IntrusiveLinkedList{Task}(nothing, nothing), Base.Threads.SpinLock(0)), (8, 140695632392064, 140695777984704)), nothing)

In [19]:
fetch(x)

2

In [20]:
y(x) = x+2

y (generic function with 1 method)

In [21]:
sendfunc(y, 2)

In [22]:
x = remotecall(y, 2, 2)

Future(2, 1, 13, ReentrantLock(nothing, 0x00000000, 0x00, Base.GenericCondition{Base.Threads.SpinLock}(Base.IntrusiveLinkedList{Task}(nothing, nothing), Base.Threads.SpinLock(0)), (0, 140695786497328, 140695773869696)), nothing)

In [23]:
fetch(x)

4

In [24]:
parentmodule(fn)

Main

In [25]:
function z(x)
    sleep(x)
    return x
end

z (generic function with 1 method)

In [26]:
sendfunc(z, 2)

In [27]:
x = remotecall(z, 2, 20)

Future(2, 1, 17, ReentrantLock(nothing, 0x00000000, 0x00, Base.GenericCondition{Base.Threads.SpinLock}(Base.IntrusiveLinkedList{Task}(nothing, nothing), Base.Threads.SpinLock(0)), (0, 140695559795968, 0)), nothing)

In [28]:
t = @async fetch(x)

Task (runnable) @0x00007ff60bdd52d0

In [29]:
while ! istaskdone(t)
    println("Task not done")
    sleep(1)
end
println("Task done")
fetch(t)

Task not done
Task not done
Task not done
Task not done
Task not done
Task not done
Task not done
Task not done
Task not done
Task not done
Task not done
Task not done
Task not done
Task not done
Task not done
Task not done
Task not done
Task not done
Task done


20